In [3]:
!pip install transformers datasets torch
!pip install datasets --upgrade
!pip install opencv-python
!pip install pymupdf scikit-learn
!pip install pdfplumber

In [6]:
!pip install keybert

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.8 MB/s eta 0:00:00


In [7]:
from transformers import RobertaTokenizer, RobertaForMaskedLM, RobertaConfig
import torch
from datasets import load_dataset
from keybert import KeyBERT
import numpy as np
import pdfplumber

In [8]:
def loadModel():
    tokenizer = RobertaTokenizer.from_pretrained("distilroberta-base")
    config = RobertaConfig.from_pretrained("distilroberta-base")
    config.gradient_checkpointing = True
    robertaModel = RobertaForMaskedLM.from_pretrained("distilroberta-base", config=config)

    return tokenizer, robertaModel

In [12]:
def keyBERTExtractor(text, top_n=10):
    kw_model = KeyBERT(model="all-MiniLM-L6-v2")
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=top_n)
    return [kw[0] for kw in keywords]

In [10]:
def trainTheModel(dataset_name="scientific_papers", config_name='arxiv', split='train', sample_size=1000):
    dataset = load_dataset(dataset_name, config_name, split="train[:500]", download_mode="force_redownload")
    texts = dataset["article"][:sample_size]
    tokenizer, model = loadModel()

    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    labels = inputs.input_ids.clone()

    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

    model.train()
    for epoch in range(3):
        optimizer.zero_grad()
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    return model

In [11]:
def extractTextFromPDF():
    pdf_path = 'sample_data/sampletext.pdf'
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # print(len(page.extract_text()))
            text += page.extract_text()
    return text

In [ ]:
text = extractTextFromPDF()
tokenizer, model = loadModel()
model = trainTheModel(config_name="arxiv")


In [13]:
keywords = keyBERTExtractor(text, top_n=10)
print("Extracted Keywords are :", keywords)

NameError: name 'text' is not defined